In [4]:
## imports
import pandas as pd
#import tensorflow as tf
from os import *
import numpy as np
import matplotlib as plt
import sklearn.model_selection as skl

In [5]:
import numpy as np

In [6]:
csv = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

df = pd.DataFrame()

wanted_cols = ['Offense Start DateTime', 'Offense Parent Group', 'Offense']

for col in wanted_cols:
    df[col] = csv[col]
    df = df.dropna()

df["Month"] = [pd.to_datetime(i).month for i in df['Offense Start DateTime']]

df["Day"] = [pd.to_datetime(i).day for i in df['Offense Start DateTime']]

df["Year"] = [pd.to_datetime(i).year for i in df['Offense Start DateTime']]

df.drop(columns = ['Offense Start DateTime'], inplace = True)

dfs = {}

KeyboardInterrupt: 

In [ ]:
def compress(frame: pd.DataFrame, wanted_columns: list, category: str) -> pd.DataFrame:
    df = pd.DataFrame()
    df[category] = frame[category]
    for col in wanted_columns:
        df[col] = frame[col]
    return df.value_counts().to_frame()

In [ ]:
for i in ['Offense Parent Group', 'Offense']:
    dfs[i] = {}
    dfs[i]["ymd"] = compress(df, ["Day", "Month", "Year"], i)
    dfs[i]["ym"] = compress(df, ["Month", "Year"], i)
    dfs[i]["y"] = compress(df, ["Year"], i)

In [ ]:
df

,Offense Parent Group,Offense,Month,Day,Year
0,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,2,5,2020
1,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,2,3,2020
2,ROBBERY,Robbery,2,2,2020
3,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,2,5,2020
4,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,2,5,2020
...,...,...,...,...,...
1033410,MOTOR VEHICLE THEFT,Motor Vehicle Theft,6,26,2013
1033411,LARCENY-THEFT,Shoplifting,2,14,2012
1033412,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,2,20,2011
1033413,LARCENY-THEFT,Shoplifting,9,19,2010


In [ ]:
df["Year"].unique()

array([2020, 2019, 2018, 2017, 2016, 2014, 2012, 2006, 2011, 2015, 2010,
       2013, 2009, 2002, 2007, 2008, 2004, 2005, 1977, 1991, 2000, 1998,
       1995, 2003, 2001, 1999, 1990, 1993, 1994, 1997, 1987, 1996, 1964,
       1992, 1980, 1908, 2021, 1979, 1915, 1929, 1974, 1984, 1989, 1953,
       1976, 1920, 1988, 2022, 1983, 2023, 1957])

In [25]:
## category is offense or offense parent category
## index for rows is years, for columns it is types
## value type is how many crimes
def one_param(category):
    l = df[[category, "Year"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tt = {}
    for i in i2c.iloc:
        if i[0] not in tt.keys():
            tt[i[0]] = {}
        else:
            if i[1] not in tt[i[0]].keys():
                tt[i[0]][i[1]] = 0
            tt[i[0]][i[1]] += int(c2i.loc[i[0]].loc[i[1]])
        
    return pd.DataFrame(tt).sort_index(ascending = False)

## multi-index dataframe
## main index is month, secondary index is year, columns are type
## value type is the percent
def two_param(category):
    l = df[[category, "Month", "Year"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tt = {}
    op = one_param(category)
    for i in i2c.iloc:
        if i[1] not in tt.keys():
            tt[i[1]] = {}
        else:
            if i[2] not in tt[i[1]]:
                tt[i[1]][i[2]] = {}
            else:
                if i[0] not in tt[i[1]][i[2]]:
                    tt[i[1]][i[2]][i[0]] = 0
                try:
                    tt[i[1]][i[2]][i[0]] += int(c2i.loc[i[0]].loc[i[1]].loc[i[2]]) / op[i[0]][i[2]]
                except:
                    pass
    tt = {k: pd.DataFrame(v) for k,v in tt.items()}
    tt = pd.concat(tt, axis=1)
    
    return tt.transpose().sort_index(ascending = True)

## returns a dictionary: keys are indexes for months, values
## are data frames:
## in each dataframe, multi-index, first index is day, second index
## is year, and value type is the percent
def three_param(category):
    ttt = {}
    l = df[[category, "Day", "Year", "Month"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tp = two_param(category)
    for m in range(1, 2):
        tt = {}
        n = l[l["Month"] == m].drop(columns = ["Month"])
        for i in i2c.iloc:
            if i[1] not in tt.keys():
                tt[i[1]] = {}
            else:
                if i[2] not in tt[i[1]]:
                    tt[i[1]][i[2]] = {}
                else:
                    if i[0] not in tt[i[1]][i[2]]:
                        tt[i[1]][i[2]][i[0]] = 0
                    try:
                        tt[i[1]][i[2]][i[0]] += float(c2i.loc[i[0]].loc[i[1]].loc[i[2]]) / tp[i[0]].iloc[i[3]].iloc[i[2]]
                    except:
                        pass
        tt = {k: pd.DataFrame(v) for k,v in tt.items()}
        tt = pd.concat(tt, axis=1)

        ttt[m] = tt.transpose().sort_index(ascending = True)
    return ttt

In [26]:
three_param("Offense")

{1:          Theft From Motor Vehicle  Burglary/Breaking & Entering  \
 1  1964                       NaN                           NaN   
    1976                       NaN                           NaN   
    1977                       NaN                           NaN   
    1980                       NaN                           NaN   
    1983                       NaN                           NaN   
 ...                           ...                           ...   
 31 2019                       0.0                           0.0   
    2020                       0.0                           0.0   
    2021                       0.0                           0.0   
    2022                       0.0                           0.0   
    2023                       0.0                           0.0   
 
          All Other Larceny  Motor Vehicle Theft  \
 1  1964                NaN                  NaN   
    1976                NaN                  NaN   
    1977               

In [11]:
opo = one_param("Offense")
opoopg = one_param("Offense Parent Group")
opoopg.to_csv("opoopg.csv")
opo.to_csv("opo.csv")


In [12]:
tpo = two_param("Offense")
tpoopg = two_param("Offense Parent Group")
tpo.to_csv("tpo.csv")
tpoopg.to_csv("tpoopg.csv")

In [13]:
thpo = three_param("Offense")
thpoopg = three_param("Offense Parent Group")

1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


In [18]:
for d in thpo:
    thpo[d].to_csv(str(d)+"thpo.csv")
    print(d)
for d in thpoopg:
    thpoopg[d].to_csv(str(d)+"thpoopg.csv")
    print(d)

1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


In [21]:
df.to_csv("df.csv")

In [20]:
thpo[1].describe()

,Theft From Motor Vehicle,Burglary/Breaking & Entering,All Other Larceny,Motor Vehicle Theft,Destruction/Damage/Vandalism of Property,Simple Assault,Shoplifting,Theft of Motor Vehicle Parts or Accessories,Robbery,Drug/Narcotic Violations,...,Animal Cruelty,Incest,"Human Trafficking, Commercial Sex Acts",Drunkenness,Bribery,Betting/Wagering,Operating/Promoting/Assisting Gambling,"Human Trafficking, Involuntary Servitude",Negligent Manslaughter,Gambling Equipment Violation
count,516.0,529.0,543.0,510.0,520.0,516.0,504.0,513.0,505.0,495.0,...,81.0,41.0,33.0,19.0,16.0,20.0,5.0,2.0,17.0,1.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(3)

In [ ]:
l[l["Month"] == m].drop(columns = ["Month"])

,Offense,Year
97,Theft From Building,2019
243,All Other Larceny,2019
327,Theft of Motor Vehicle Parts or Accessories,2019
372,Credit Card/Automated Teller Machine Fraud,2019
396,Fondling,2019
...,...,...
1033211,Driving Under the Influence,2021
1033212,Stolen Property Offenses,2021
1033213,Weapon Law Violations,2021
1033286,Sexual Assault With An Object,2020


In [ ]:
one_param("Offense")

,Theft From Motor Vehicle,Burglary/Breaking & Entering,Identity Theft,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,All Other Larceny,Theft of Motor Vehicle Parts or Accessories,Simple Assault,Shoplifting,Trespass of Real Property,...,"Human Trafficking, Commercial Sex Acts",Drunkenness,Justifiable Homicide,Incest,Negligent Manslaughter,Betting/Wagering,Bribery,"Human Trafficking, Involuntary Servitude",Operating/Promoting/Assisting Gambling,Gambling Equipment Violation
2023,834.0,886.0,76.0,754.0,862.0,577.0,572.0,554.0,339.0,200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,8927.0,8666.0,751.0,7236.0,NaN,NaN,NaN,5074.0,4323.0,2047.0,...,12.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN
2021,9333.0,9823.0,823.0,NaN,5329.0,6239.0,5709.0,4683.0,3949.0,1825.0,...,8.0,5.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
2020,9140.0,NaN,NaN,6658.0,4942.0,6102.0,3150.0,4755.0,3166.0,2624.0,...,8.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2019,10757.0,7643.0,660.0,5804.0,3991.0,6667.0,1405.0,5593.0,4523.0,3914.0,...,NaN,3.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN
2018,11714.0,8105.0,104.0,6651.0,4099.0,6560.0,1458.0,NaN,NaN,NaN,...,NaN,NaN,2.0,5.0,NaN,1.0,1.0,NaN,1.0,NaN
2017,12516.0,7790.0,48.0,6644.0,3705.0,5935.0,1393.0,5630.0,4767.0,3909.0,...,2.0,NaN,3.0,2.0,NaN,3.0,2.0,NaN,NaN,NaN
2016,13191.0,7549.0,35.0,6322.0,4175.0,5569.0,1430.0,5475.0,4809.0,2832.0,...,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
2015,12683.0,7593.0,22.0,5983.0,3873.0,6144.0,1170.0,5278.0,4423.0,2189.0,...,NaN,NaN,2.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN
2014,NaN,7135.0,16.0,5750.0,5497.0,6358.0,1529.0,5060.0,4304.0,1367.0,...,NaN,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN


In [ ]:
two_param("Offense Parent Group")


ASSAULT OFFENSES  BURGLARY/BREAKING&ENTERING  \
1  1964               NaN                         NaN   
   1977               NaN                         NaN   
   1980               NaN                         NaN   
   1983               NaN                         NaN   
   1987               NaN                         NaN   
...                   ...                         ...   
12 2018               0.0                         0.0   
   2019               0.0                         0.0   
   2020               0.0                         0.0   
   2021               0.0                         0.0   
   2022               0.0                         0.0   

         DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY  MOTOR VEHICLE THEFT  \
1  1964                                       NaN                  NaN   
   1977                                       NaN                  NaN   
   1980                                       NaN                  NaN   
   1983                                       NaN                  NaN   
   1987                                       NaN                  NaN   
...                                           ...                  ...   
12 2018                                       0.0                  0.0   
   2019                                       0.0                  0.0   
   2020                                       0.0                  0.0   
   2021                                       0.0                  0.0   
   2022                                       0.0                  0.0   

         FRAUD OFFENSES  ROBBERY  TRESPASS OF REAL PROPERTY  \
1  1964             NaN      NaN                        NaN   
   1977             NaN      NaN                        NaN   
   1980             NaN      NaN                        NaN   
   1983             NaN      NaN                        NaN   
   1987             NaN      NaN                        NaN   
...                 ...      ...                        ...   
12 2018             0.0      0.0                        0.0   
   2019             0.0      0.0                        0.0   
   2020             0.0      0.0                        0.0   
   2021             0.0      0.0                        0.0   
   2022             0.0      0.0                        0.0   

         DRUG/NARCOTIC OFFENSES  FAMILY OFFENSES, NONVIOLENT  \
1  1964                     NaN                          NaN   
   1977                     NaN                          NaN   
   1980                     NaN                          NaN   
   1983                     NaN                          NaN   
   1987                     NaN                          NaN   
...                         ...                          ...   
12 2018                     0.0                          0.0   
   2019                     0.0                          0.0   
   2020                     0.0                          0.0   
   2021                     0.0                          0.0   
   2022                     0.0                          0.0   

         DRIVING UNDER THE INFLUENCE  ...  EXTORTION/BLACKMAIL  \
1  1964                          NaN  ...                  NaN   
   1977                          NaN  ...                  NaN   
   1980                          NaN  ...                  NaN   
   1983                          NaN  ...                  NaN   
   1987                          NaN  ...                  NaN   
...                              ...  ...                  ...   
12 2018                          0.0  ...                  0.0   
   2019                          0.0  ...                  0.0   
   2020                          0.0  ...                  0.0   
   2021                          0.0  ...                  0.0   
   2022                          0.0  ...                  0.0   

         PORNOGRAPHY/OBSCENE MATERIAL  HOMICIDE OFFENSES  \
1  1964                           NaN                NaN

In [ ]:
compress(df, ["Year", "Month", "Day"], "Offense")

0
Offense                  Year Month Day     
Identity Theft           2020 5     18   585
                              3     8    516
                              5     15   500
                                    14   453
                                    11   406
...                                      ...
Fondling                 2018 1     29     1
Stolen Property Offenses 2022 6     20     1
Fondling                 2018 1     23     1
Stolen Property Offenses 2022 6     22     1
Aggravated Assault       1997 3     12     1

[139112 rows x 1 columns]

In [ ]:
df.value_counts().to_frame()

0
Offense Parent Group        Offense                     Month Day Year     
FRAUD OFFENSES              Identity Theft              5     18  2020  585
                                                        3     8   2020  516
                                                        5     15  2020  500
                                                              14  2020  453
                                                              11  2020  406
...                                                                     ...
FAMILY OFFENSES, NONVIOLENT Family Offenses, Nonviolent 7     8   2010    1
                                                              7   2018    1
                                                                  2016    1
                                                                  2015    1
ANIMAL CRUELTY              Animal Cruelty              1     1   2023    1

[139112 rows x 1 columns]

In [ ]:
csv.columns

Index(['Report Number', 'Offense ID', 'Offense Start DateTime',
       'Offense End DateTime', 'Report DateTime', 'Group A B',
       'Crime Against Category', 'Offense Parent Group', 'Offense',
       'Offense Code', 'Precinct', 'Sector', 'Beat', 'MCPP',
       '100 Block Address', 'Longitude', 'Latitude'],
      dtype='object')

In [ ]:
df['Offense Start DateTime'].isna().value_counts()

False    1032212
Name: Offense Start DateTime, dtype: int64

In [ ]:
## datasets, data cleanup, test-train split
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

## cleaning up the dates to have month, day, year columns
## also splitting it up into the layers of data
## which includes year, month, day

ds = ds[ds['Offense Start DateTime'] != None]

# dateTime = ds['Offense Start DateTime'].dropna()

ds["Month"] = [pd.to_datetime(i).month for i in ds['Offense Start DateTime']]

ds["Day"] = [pd.to_datetime(i).day for i in ds['Offense Start DateTime']]

ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

ds.loc[:, 'Crime Against Category':'Year']

,Crime Against Category,Offense Parent Group,Offense,Offense Code,Precinct,Sector,Beat,MCPP,100 Block Address,Longitude,Latitude,Month,Day,Year
0,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,Q1,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387,2.0,5.0,2020.0
1,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118,2.0,3.0,2020.0
2,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552,47.666384,2.0,2.0,2020.0
3,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384865,47.642927,2.0,5.0,2020.0
4,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,B2,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195,47.662193,2.0,5.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033410,PROPERTY,MOTOR VEHICLE THEFT,Motor Vehicle Theft,240,W,Q,Q3,QUEEN ANNE,1ST AVE N / VALLEY ST,-122.355386,47.626251,6.0,26.0,2013.0
1033411,PROPERTY,LARCENY-THEFT,Shoplifting,23C,N,U,U1,ROOSEVELT/RAVENNA,73XX BLOCK OF ROOSEVELT WAY NE,-122.317464,47.681894,2.0,14.0,2012.0
1033412,PROPERTY,BURGLARY/BREAKING&ENTERING,Burglary/Breaking & Entering,220,S,S,S3,RAINIER VIEW,100XX BLOCK OF M L KING JR WAY S,-122.278352,47.510453,2.0,20.0,2011.0
1033413,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,M,M3,DOWNTOWN COMMERCIAL,5XX BLOCK OF PIKE ST,-122.335018,47.610909,9.0,19.0,2010.0


In [ ]:
## train-test split
## to-do: for dsYear, Month, Day, create 3 splits
## each of 3 splits will be split 75% train and 25% test

In [ ]:
## polynomial regression
## to-do: learn how to do polynomial regression ,_,

In [ ]:
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

In [ ]:
ds['Offense Start DateTime'].dropna()

0          02/05/2020 10:10:00 AM
1          02/03/2020 08:00:00 AM
2          02/02/2020 08:30:00 PM
3          02/05/2020 01:17:00 AM
4          02/05/2020 12:51:21 AM
                    ...          
1033410    06/26/2013 11:00:00 AM
1033411    02/14/2012 03:04:00 PM
1033412    02/20/2011 08:00:00 PM
1033413    09/19/2010 04:59:00 PM
1033414    02/25/2010 06:00:00 PM
Name: Offense Start DateTime, Length: 1032212, dtype: object

In [ ]:
ds['Report DateTime'].isnull().value_counts()

False    1033415
Name: Report DateTime, dtype: int64

In [ ]:
ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

In [27]:
opo = pd.read_csv("opo.csv")

In [29]:
tpo = pd.read_csv("tpo.csv")

In [31]:
tpo_dict = {}

In [69]:
pd.read_csv("opo.csv")[["Unnamed: 0", "Burglary/Breaking & Entering"]]

,Unnamed: 0,Burglary/Breaking & Entering
0,2023,886.0
1,2022,8666.0
2,2021,9823.0
3,2020,NaN
4,2019,7643.0
5,2018,8105.0
6,2017,7790.0
7,2016,7549.0
8,2015,7593.0
9,2014,7135.0


In [68]:
pd.read_csv("9thpo.csv")

def a(df, df2, crime):
    df = pd.read_csv(df)
    n = df[["Unnamed: 0", "Unnamed: 1", crime]]
    dict1 = {}
    for i in range(1, 31):
        try:
            n = n[n["Unnamed: 0"] == i]
            m = n[["Unnamed: 1", crime]] / pd.read_csv(df2)[["Unnamed: 0", crime]]
            dict1[i] = m.describe().mean()[crime]
        except:
            pass
    return dict1

a("7thpo.csv", "opo.csv", "Burglary/Breaking & Entering")

{1: 0.625,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0}

In [65]:
pd.read_csv("9thpo.csv")

,Unnamed: 0,Unnamed: 1,Theft From Motor Vehicle,Burglary/Breaking & Entering,All Other Larceny,Motor Vehicle Theft,Destruction/Damage/Vandalism of Property,Simple Assault,Shoplifting,Theft of Motor Vehicle Parts or Accessories,...,Animal Cruelty,Incest,"Human Trafficking, Commercial Sex Acts",Drunkenness,Bribery,Betting/Wagering,Operating/Promoting/Assisting Gambling,"Human Trafficking, Involuntary Servitude",Negligent Manslaughter,Gambling Equipment Violation
0,1,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,31,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,31,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
740,31,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,31,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
def a(df, df2, crime):
    df = pd.read_csv(df)
    n = df[["Unnamed: 0", "Unnamed: 1", crime]]
    dict1 = {}
    for i in range(1, 31):
        try:
            n = n[n["Unnamed: 0"] == i]
            m = n[["Unnamed: 1", crime]] / pd.read_csv(df2)[["Unnamed: 0", crime]]
            dict1[i] = m.describe().mean()[crime]
        except:
            pass
    return dict1

def average(df, crime):
    if (df.endswith("o.csv")):
        return a(df, "opo.csv", crime)
    elif (df.endswith("g.csv")):
        return a(df, "opoopg.csv", crime)
    df = pd.read_csv(df)
    n = df[["Unnamed: 0", "Unnamed: 1", crime]]
    dict1 = {}
    for i in range(1, 13):
        dict1[i] = n[n["Unnamed: 0"] == i].describe().mean()[crime]
    return dict1
    

In [53]:
pd.read_csv("opo.csv")

,Unnamed: 0,Theft From Motor Vehicle,Burglary/Breaking & Entering,Identity Theft,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,All Other Larceny,Theft of Motor Vehicle Parts or Accessories,Simple Assault,Shoplifting,...,"Human Trafficking, Commercial Sex Acts",Drunkenness,Justifiable Homicide,Incest,Negligent Manslaughter,Betting/Wagering,Bribery,"Human Trafficking, Involuntary Servitude",Operating/Promoting/Assisting Gambling,Gambling Equipment Violation
0,2023,834.0,886.0,76.0,754.0,862.0,577.0,572.0,554.0,339.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,8927.0,8666.0,751.0,7236.0,NaN,NaN,NaN,5074.0,4323.0,...,12.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN
2,2021,9333.0,9823.0,823.0,NaN,5329.0,6239.0,5709.0,4683.0,3949.0,...,8.0,5.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
3,2020,9140.0,NaN,NaN,6658.0,4942.0,6102.0,3150.0,4755.0,3166.0,...,8.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
4,2019,10757.0,7643.0,660.0,5804.0,3991.0,6667.0,1405.0,5593.0,4523.0,...,NaN,3.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN
5,2018,11714.0,8105.0,104.0,6651.0,4099.0,6560.0,1458.0,NaN,NaN,...,NaN,NaN,2.0,5.0,NaN,1.0,1.0,NaN,1.0,NaN
6,2017,12516.0,7790.0,48.0,6644.0,3705.0,5935.0,1393.0,5630.0,4767.0,...,2.0,NaN,3.0,2.0,NaN,3.0,2.0,NaN,NaN,NaN
7,2016,13191.0,7549.0,35.0,6322.0,4175.0,5569.0,1430.0,5475.0,4809.0,...,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,12683.0,7593.0,22.0,5983.0,3873.0,6144.0,1170.0,5278.0,4423.0,...,NaN,NaN,2.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN
9,2014,NaN,7135.0,16.0,5750.0,5497.0,6358.0,1529.0,5060.0,4304.0,...,NaN,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN


In [52]:
average("opo.csv", "Burglary/Breaking & Entering")

KeyError: "['Unnamed: 1'] not in index"

In [51]:
average("8thpo.csv", "Burglary/Breaking & Entering")

{1: 2.375,
 2: 2.0,
 3: 2.125,
 4: 2.25,
 5: 2.125,
 6: 2.0,
 7: 2.125,
 8: 2.125,
 9: 2.0,
 10: 2.125,
 11: 2.375,
 12: 2.0}

In [39]:
n = tpo[["Unnamed: 0", "Unnamed: 1", "Burglary/Breaking & Entering"]]

In [44]:
n[n["Unnamed: 0"] == 1].describe().mean()["Burglary/Breaking & Entering"]

1.8478589907274814

In [ ]:
for i in tpo.iloc:
    if 

In [32]:
tpo

,Unnamed: 0,Unnamed: 1,Burglary/Breaking & Entering,All Other Larceny,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,Simple Assault,Shoplifting,Aggravated Assault,Intimidation,...,Theft From Motor Vehicle,Hacking/Computer Invasion,Animal Cruelty,Purchasing Prostitution,Drunkenness,"Human Trafficking, Commercial Sex Acts",Bribery,Operating/Promoting/Assisting Gambling,Gambling Equipment Violation,"Human Trafficking, Involuntary Servitude"
0,1,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,12,2018,0.088957,0.098018,0.082845,0.081971,NaN,NaN,0.076477,0.073925,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
346,12,2019,0.085438,0.085346,0.078222,0.086194,0.079027,0.077161,0.087068,0.086123,...,NaN,0.140351,0.117647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,12,2020,NaN,0.117666,0.080505,0.092473,0.068980,0.086229,0.073949,0.076469,...,0.082604,0.076923,0.115385,0.047619,NaN,NaN,NaN,NaN,NaN,NaN
348,12,2021,NaN,0.087193,NaN,0.100206,0.079863,0.092682,0.080145,NaN,...,0.090646,NaN,NaN,0.133333,NaN,0.125,NaN,NaN,NaN,NaN


In [ ]:
## polynomial regression

# split the data 50/50

# for i in range 10
#     create the model 
#     calculate mse for 1/2 and 2/2
#     if abs(mse1-mse2) < curmin curmin = abs(mse1-mse2) and bestmodel = curmodel 

# settle on the last model

#------------LOOK AT THIS CODE!!!------------#
# loop through offenses

X = list(one_param("Offense").index)
y = list(one_param("Offense")[off].values())

X1, X2, y1, y2 = skl.train_test_split(X, y, test_size=0.5, train_size=0.5, random_state=None, shuffle=True, stratify=None)

# X1 = []  # for years 1/2
# y1 = []  # for occurences in that year 1/2

# X2 = []  # for years 2/2
# y2 = []  # for occurences in that year 2/2

bestmodel = None
curmin = int("inf")

for i in range(1, 11):
    curmodel = PolynomialRegression(degree = i)
    curmodel.fit(X1, y1)
    msetrain = mse(y1, curmodel.predict(X1))
    msetest = mse(y2, curmodel.predict(X2))

    diff = abs(msetrain-msetest)

    if diff < curmin:
        curmin = diff
        bestmodel = curmodel  

plt.scatter(X1, y1, color = 'r')
plt.plot(X2, y2, color = 'b')
plt.plot(X1, bestmodel.predict(X1), color = 'g')

#--------------------------------------------#

In [ ]:
def predictCrime(crimeName, year, month, day):
    X = list(one_param("Offense").index)
    y = list(one_param("Offense")[crimeName].values())

    X1, X2, y1, y2 = skl.train_test_split(X, y, test_size=0.5, train_size=0.5, random_state=None, shuffle=True, stratify=None)

    bestmodel = None
    curmin = int("inf")
    marginoferror = 0

    for i in range(1, 11):
        curmodel = PolynomialRegression(degree = i)
        curmodel.fit(X1, y1)
        predictX1 = curmodel.predict(X1)
        predictX2 = curmodel.predict(X2)

        msetrain = mse(y1, predictX1)
        msetest = mse(y2, predictX2)

        diff = abs(msetrain-msetest)

        if diff < curmin:
            curmin = diff
            bestmodel = curmodel  
            marginoferror = 1-skl.mean_squared_error(y2, predictX2, *, sample_weight=None, multioutput='uniform_average', squared=True)

    res = bestmodel.predict(year)
    res = res * #monthavg * dayavg

    return round(res), marginoferror